After the exploration of different models in the Model_Exploration notebook, we have settled on our final model. This model is a Beta Regression using 100 attributes.

Below we train this model and we also have a playground to see how altering different attributes will modify the mobility probability.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from patsy import dmatrix, dmatrices
import sklearn.metrics as metrics
from distfit import distfit
import scipy.stats as stats
import itertools
from sklearn.feature_selection import SelectKBest, f_classif
import statsmodels.api as sm
from sklearn.model_selection import KFold, train_test_split
from tqdm.auto import tqdm
from statsmodels.othermod.betareg import BetaModel
from sklearn.ensemble import RandomForestRegressor
import pickle
import ipywidgets as widgets
from IPython.display import display

In [2]:
income_college = pd.read_csv('mrc_table2.csv')
college_characteristics = pd.read_csv('mrc_table10.csv')
cols_to_use = list(college_characteristics.columns.difference(income_college.columns)) + ['super_opeid']
df= income_college.merge(college_characteristics[cols_to_use], on='super_opeid')
df["region"] = df["region"].astype(int)

region_mapping = df[["state", "region"]].drop_duplicates().set_index("state")['region'].to_dict()

In [3]:
BUFFER = 0.000001

In [4]:
formula = """alien_share_fall_2000 + asian_or_pacific_share_fall_2000 + avgfacsal_2013 +
             black_share_fall_2000 + np.log10(exp_instr_2012) + flagship + hbcu +
             hisp_share_fall_2000 + pct_arthuman_2000 + pct_business_2000 + pct_health_2000 +
             pct_multidisci_2000 + pct_publicsocial_2000 + pct_socialscience_2000 +
             pct_stem_2000 + pct_tradepersonal_2000 + C(region) + scorecard_median_earnings_2011 +
             C(state) + sticker_price_2013 + C(tier_name)"""

In [5]:
def modify_features(data):
    """Given an input dataframe, this does some feature engineering to create features of interest.
    Mostly, these are feature interactions."""

    data = data.copy()

    # Keenly interested in how the majors and racial breakdowwns interact w/ each other moreso than raw numbers
    # Does a school with heavy stem, but light business do better than stem alone, etc.
    majors_and_shares = data.columns[
        (data.columns.str.contains("pct_") | data.columns.str.contains("share_")) & 
        (~data.columns.str.contains("_X_")) & (~data.columns.str.contains("_DIV_"))
    ]
    majors_and_shares = sorted(majors_and_shares)
    data[majors_and_shares] = data[majors_and_shares].fillna(0)
    out = {}
    for i, c1 in enumerate(majors_and_shares):
        for j, c2 in enumerate(majors_and_shares):
            if j > i:
                out[f"{c1}_X_{c2}"] = data[c1] * data[c2]
                out[f"{c1}_DIV_{c2}"] = (data[c1] / (data[c1] + data[c2])).fillna(0)
    out = pd.DataFrame(out)
    to_drop = [col for col in out.columns if col in data.columns]        
    data = data.drop(to_drop, axis=1).join(out, how='outer')

    # The barrons score we modify to make it more linear, in case that is useful to the model
    if "barrons" in data.columns:
        data['mod_barrons'] = data.barrons.apply(lambda x: x if x != 999 else 10)

    # Finally we also generate a ratio of faculty salary to sticker price
    data['avgfacsal_2013_DIV_sticker_price_2013'] = data.avgfacsal_2013 / data.sticker_price_2013

    return data

In [6]:
# See the model_exploration notebook for how these wwere chosen
selected_cols = [
       'Intercept',
       'C(region)[T.3]', 'C(state)[T.AL]', 'C(state)[T.AR]',
       'C(state)[T.AZ]', 'C(state)[T.CA]', 'C(state)[T.CT]',
       'C(state)[T.DC]', 'C(state)[T.DE]', 'C(state)[T.FL]',
       'C(state)[T.GA]', 'C(state)[T.HI]', 'C(state)[T.IA]',
       'C(state)[T.ID]', 'C(state)[T.IL]', 'C(state)[T.IN]',
       'C(state)[T.KY]', 'C(state)[T.LA]', 'C(state)[T.MA]',
       'C(state)[T.MD]', 'C(state)[T.ME]', 'C(state)[T.MI]',
       'C(state)[T.MO]', 'C(state)[T.MS]', 'C(state)[T.MT]',
       'C(state)[T.NC]', 'C(state)[T.ND]', 'C(state)[T.NE]',
       'C(state)[T.NH]', 'C(state)[T.NJ]', 'C(state)[T.NM]',
       'C(state)[T.NV]', 'C(state)[T.OH]', 'C(state)[T.OK]',
       'C(state)[T.OR]', 'C(state)[T.RI]', 'C(state)[T.SC]',
       'C(state)[T.SD]', 'C(state)[T.TN]', 'C(state)[T.TX]',
       'C(state)[T.VA]', 'C(state)[T.VT]', 'C(state)[T.WA]',
       'C(state)[T.WI]', 'C(state)[T.WV]', 'C(state)[T.WY]',
       'C(tier_name)[T.Highly selective private]',
       'C(tier_name)[T.Highly selective public]',
       'C(tier_name)[T.Ivy Plus]',
       'C(tier_name)[T.Nonselective four-year private not-for-profit]',
       'C(tier_name)[T.Nonselective four-year public]',
       'C(tier_name)[T.Other elite schools (public and private)]',
       'C(tier_name)[T.Selective public]', 'hbcu', 'flagship',
       'sticker_price_2013', 'scorecard_median_earnings_2011',
       'np.log10(exp_instr_2012)', 'hisp_share_fall_2000',
       'pct_stem_2000', 'pct_tradepersonal_2000',
       'asian_or_pacific_share_fall_2000_X_black_share_fall_2000',
       'asian_or_pacific_share_fall_2000_X_hisp_share_fall_2000',
       'alien_share_fall_2000_X_asian_or_pacific_share_fall_2000',
       'asian_or_pacific_share_fall_2000_X_pct_arthuman_2000',
       'asian_or_pacific_share_fall_2000_X_pct_multidisci_2000',
       'asian_or_pacific_share_fall_2000_X_pct_publicsocial_2000',
       'asian_or_pacific_share_fall_2000_X_pct_stem_2000',
       'asian_or_pacific_share_fall_2000_X_pct_socialscience_2000',
       'asian_or_pacific_share_fall_2000_X_pct_tradepersonal_2000',
       'alien_share_fall_2000_X_black_share_fall_2000',
       'black_share_fall_2000_X_pct_stem_2000',
       'black_share_fall_2000_X_pct_socialscience_2000',
       'hisp_share_fall_2000_X_pct_arthuman_2000',
       'hisp_share_fall_2000_X_pct_business_2000',
       'hisp_share_fall_2000_X_pct_multidisci_2000',
       'hisp_share_fall_2000_X_pct_stem_2000',
       'hisp_share_fall_2000_X_pct_socialscience_2000',
       'hisp_share_fall_2000_X_pct_tradepersonal_2000',
       'alien_share_fall_2000_X_pct_publicsocial_2000',
       'alien_share_fall_2000_X_pct_stem_2000',
       'alien_share_fall_2000_X_pct_socialscience_2000',
       'alien_share_fall_2000_X_pct_tradepersonal_2000',
       'pct_arthuman_2000_X_pct_multidisci_2000',
       'pct_arthuman_2000_X_pct_publicsocial_2000',
       'pct_arthuman_2000_X_pct_stem_2000',
       'pct_arthuman_2000_X_pct_socialscience_2000',
       'pct_arthuman_2000_X_pct_tradepersonal_2000',
       'pct_business_2000_X_pct_socialscience_2000',
       'pct_health_2000_X_pct_multidisci_2000',
       'pct_health_2000_X_pct_tradepersonal_2000',
       'pct_multidisci_2000_X_pct_publicsocial_2000',
       'pct_multidisci_2000_X_pct_stem_2000',
       'pct_multidisci_2000_X_pct_socialscience_2000',
       'pct_multidisci_2000_X_pct_tradepersonal_2000',
       'pct_publicsocial_2000_X_pct_stem_2000',
       'pct_publicsocial_2000_X_pct_socialscience_2000',
       'pct_socialscience_2000_X_pct_stem_2000',
       'pct_stem_2000_X_pct_tradepersonal_2000',
       'pct_socialscience_2000_X_pct_tradepersonal_2000',
       'avgfacsal_2013_DIV_sticker_price_2013'
]

In [7]:
def prep_data(df, mode="half"):
    if mode == "full":
        form_set = dmatrices("mr_kq5_pq1 ~ " + formula, df)
        formed = form_set[1]
    formed = dmatrix(formula, df)
    X = pd.DataFrame(formed)
    X.columns = formed.design_info.column_name_indexes
    data = X
    data = modify_features(data)
    
    cols = [col for col in data.columns if col in selected_cols]
    data = data[cols]
    if mode == "full":
        assert len(set(selected_cols) - set(data.columns)) == 0
    else:
        for col in (set(selected_cols) - set(data.columns)):
            data[col] = 0

    data = data[sorted(data.columns)]
    
    if mode == "full":
        data["output"] = form_set[0] + BUFFER
    
    return data

In [8]:
data = prep_data(df, "full")
X = data.drop("output", axis=1)
y = data.output

model = BetaModel(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              BetaModel Results                               
==============================================================================
Dep. Variable:                 output   Log-Likelihood:                 7166.6
Model:                      BetaModel   AIC:                        -1.413e+04
Method:            Maximum Likelihood   BIC:                        -1.356e+04
Date:                Thu, 09 May 2024                                         
Time:                        13:09:15                                         
No. Observations:                2007                                         
Df Residuals:                    1905                                         
Df Model:                         100                                         
=================================================================================================================================
                                                                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------
C(region)[T.3]                                                    0.0600      8.664      0.007      0.994     -16.922      17.042
C(state)[T.AL]                                                    0.1774      8.664      0.020      0.984     -16.804      17.158
C(state)[T.AR]                                                    0.2115      8.664      0.024      0.981     -16.769      17.192
C(state)[T.AZ]                                                   -0.1078      0.109     -0.992      0.321      -0.321       0.105
C(state)[T.CA]                                                   -0.0338      0.059     -0.578      0.563      -0.149       0.081
C(state)[T.CT]                                                    0.0007      0.081      0.009      0.993      -0.158       0.160
C(state)[T.DC]                                                   -0.3002      8.657     -0.035      0.972     -17.268      16.667
C(state)[T.DE]                                                   -0.2074      8.667     -0.024      0.981     -17.194      16.779
C(state)[T.FL]                                                    0.0036      8.661      0.000      1.000     -16.972      16.980
C(state)[T.GA]                                                   -0.1367      8.663     -0.016      0.987     -17.116      16.842
C(state)[T.HI]                                                   -0.5435      0.297     -1.831      0.067      -1.125       0.038
C(state)[T.IA]                                                   -0.0650      0.082     -0.798      0.425      -0.225       0.095
C(state)[T.ID]                                                    0.1231      0.146      0.842      0.400      -0.164       0.410
C(state)[T.IL]                                                   -0.1563      0.055     -2.840      0.005      -0.264      -0.048
C(state)[T.IN]                                                   -0.3116      0.094     -3.321      0.001      -0.496      -0.128
C(state)[T.KY]                                                    0.0399      8.666      0.005      0.996     -16.946      17.026
C(state)[T.LA]                                                    0.3160      8.665      0.036      0.971     -16.668      17.300
C(state)[T.MA]                                                    0.0222      0.055      0.404      0.686      -0.085       0.130
C(state)[T.MD]                                                   -0.0668      8.663     -0.008      0.994     -17.046      16.912
C(state)[T.ME]                                                   -0.0239      0.154     -0.155      0.877      -0.326       0.278
C(state)[T.MI]                                                   -0.1730      0.067     -2.567      0.010      -0.305      -0.041
C(state)[T.MO]                            

In [9]:
# Start with the data for Columbia
initial = df.loc[df.name.str.contains("Columbia University")].iloc[0]

def recalculate(change):
    """Runs the model given our inputs"""
    data = {}
    for share, col in zip(shares, share_mappings):
        val = float(share.value)
        if "share" in col:
            val /= 100.0
        data[col] = val
    data["flagship"] = flagship_university_checkbox.value
    data["hbcu"] = hbcu_checkbox.value
    data["exp_instr_2012"] = instructional_expenditures.value
    data["sticker_price_2013"] = sticker_price.value
    data["avgfacsal_2013"] = faculty_salary.value
    data["scorecard_median_earnings_2011"] = earnings.value
    data["state"] = state_dropdown.value
    data["region"] = region_mapping[data["state"]]
    data["tier_name"] = tier_dropdown.value
    
    t = pd.Series(data).to_frame()
    t[1] = -1
    t = t[[1,0]]
    t = t.T
    for col in t.columns:
        if col in ("state", "region", "tier_name"):
            continue
        t[col] = t[col].astype(float)
    
    t = prep_data(t)
    prediction = model.predict(t)[0]
    
    s = f"{prediction:.2%}"
    result_text.value = s

# Create a list of sliders for various shares
share_descriptions = [
    "Non-US Citizens", "AAPI Students", "Black Students", "Hispanic Students",
    "Arts & Humanities Majors", "Business Majors", "Health Majors", 
    "Public & Social Services Majors", "Social Sciences Majors", "STEM Majors", 
    "Trades & Professional Majors", "Multidisciplinary Majors"
]
share_mappings = ['alien_share_fall_2000', 'asian_or_pacific_share_fall_2000',
                  'black_share_fall_2000','hisp_share_fall_2000','pct_arthuman_2000',
                  'pct_business_2000','pct_health_2000', 'pct_publicsocial_2000',
                  'pct_socialscience_2000','pct_stem_2000','pct_tradepersonal_2000',
                  'pct_multidisci_2000']
shares = [
    widgets.FloatSlider(description=f'{desc} (%):', min=0, max=100, step=0.1,
                        value=(initial[col] * (100 if 'share' in col else 1)),
                        layout={'width': 'auto'}, style={'description_width':"initial"})
    for desc, col in zip(share_descriptions, share_mappings)
]

# Checkboxes for university types
flagship_university_checkbox = widgets.Checkbox(description='Is Flagship University', value=bool(initial.flagship),
                                                style={'description_width': 'initial'})
hbcu_checkbox = widgets.Checkbox(description='Is HBCU', value=bool(initial.hbcu), style={'description_width': 'initial'})

# Numeric entries for dollar values
instructional_expenditures = widgets.IntText(description='Total Instructional Expenditures ($):', 
                                             value=initial.exp_instr_2012,
                                             layout={"width":"600px"},
                                             style={'description_width': 'initial'})
sticker_price = widgets.IntText(description='Sticker Price ($):', 
                                value=initial.sticker_price_2013,
                                layout={"width":"600px"},
                                style={'description_width': 'initial'})
faculty_salary = widgets.IntText(description='Average Faculty Salary ($):', 
                                 value=initial.avgfacsal_2013,
                                 layout={"width":"600px"},                                 
                                 style={'description_width': 'initial'})
earnings = widgets.IntText(description='Median Earnings of Graduates ($):',
                           layout={"width":"600px"},
                           value=initial.scorecard_median_earnings_2011,
                           style={'description_width': 'initial'})

# Dropdown for US state codes
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
state_dropdown = widgets.Dropdown(options=states, value=initial.state, 
                                  description='State:', style={'description_width': 'initial'})

# Dropdown for tiers
tiers = [
    "Ivy Plus", "Other elite schools (public and private)", "Highly selective public",
    "Highly selective private", "Selective public", "Selective private", "Nonselective 4-year public",
    "Nonselective 4-year private not-for-profit", "Two-year (public and private not-for-profit)",
    "Four-year for-profit", "Two-year for-profit", "Less than two year schools of any type"
]
tier_dropdown = widgets.Dropdown(options=tiers, value=initial.tier_name,
                                 description='Tier:', style={'description_width': 'initial'})

# Result display textbox
result_text = widgets.Text(description='Mobility Probability:', style={'description_width': 'initial'}, layout={'width': '500px'})

# Add observers to each widget to trigger recalculation
for share in shares:
    share.observe(recalculate, 'value')
flagship_university_checkbox.observe(recalculate, 'value')
hbcu_checkbox.observe(recalculate, 'value')
instructional_expenditures.observe(recalculate, 'value')
sticker_price.observe(recalculate, 'value')
faculty_salary.observe(recalculate, 'value')
earnings.observe(recalculate, 'value')
state_dropdown.observe(recalculate, 'value')
tier_dropdown.observe(recalculate, 'value')

# Display widgets
display(*shares, flagship_university_checkbox, hbcu_checkbox, instructional_expenditures,
        sticker_price, faculty_salary, earnings, state_dropdown, tier_dropdown, result_text)

FloatSlider(value=13.906085000000001, description='Non-US Citizens (%):', layout=Layout(width='auto'), style=S…

FloatSlider(value=12.667975, description='AAPI Students (%):', layout=Layout(width='auto'), style=SliderStyle(…

FloatSlider(value=7.322965600000001, description='Black Students (%):', layout=Layout(width='auto'), style=Sli…

FloatSlider(value=6.719009600000001, description='Hispanic Students (%):', layout=Layout(width='auto'), style=…

FloatSlider(value=24.198616, description='Arts & Humanities Majors (%):', layout=Layout(width='auto'), style=S…

...

IntText(value=15706, description='Average Faculty Salary ($):', layout=Layout(width='600px'), style=Descriptio…

IntText(value=72900, description='Median Earnings of Graduates ($):', layout=Layout(width='600px'), style=Desc…

Dropdown(description='State:', index=31, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', …

Dropdown(description='Tier:', options=('Ivy Plus', 'Other elite schools (public and private)', 'Highly selecti…

Text(value='', description='Mobility Probability:', layout=Layout(width='500px'), style=TextStyle(description_…